### Filtering manually and statistically (Step 1)
manual deletions in meshlab might still be better

In [4]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
# Step 1: Load the PLY file
point_cloud = o3d.io.read_point_cloud("dense_maze2_vid1.0.ply") #week 1 maze
print('succeeded')

succeeded


In [4]:
o3d.visualization.draw_geometries([point_cloud]) 

In [5]:
"""visualise"""
# Add axis
axpoints = o3d.utility.Vector3dVector([[0, 0, 0], [1, 0, 0], [0, 1, 0]])
axis = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=10, origin=[0, 0, 0]
)

# Maze coord system: X-red, Y-green, Z-blue
# Global coord system: X-blue, Y-red, Z-green
#Show
o3d.visualization.draw_geometries([point_cloud,axis]) 

: 

In [10]:
"""Transform to np and inspect"""
point_cloud_np = np.asarray(point_cloud.points) 
print('Shape: ', point_cloud_np.shape)
print('First point: ', point_cloud_np[0, :])


Shape:  (1354969, 3)
First point:  [  22.64030075 -388.94900513  744.36401367]


Analysis

In [7]:
print("X")
x=point_cloud_np[:, 2]
print("mean",np.mean(x))
print("std", np.std(x))
print("max",np.max(x))
print("min",np.min(x))
print("")

print("Y")
y=point_cloud_np[:, 0]
print("mean",np.mean(y))
print("std", np.std(y))
print("max",np.max(y))
print("min",np.min(y))
print("")


print("Z")
z=point_cloud_np[:, 1]
print("mean",np.mean(z))
print("std", np.std(z))
print("max",np.max(z))
print("min",np.min(z))


X
mean 771.9247780269158
std 255.2889864492842
max 2722.449951171875
min -799.7979736328125

Y
mean -278.9846202153612
std 241.92960301952183
max 1145.6300048828125
min -1862.3800048828125

Z
mean -65.69069134097202
std 120.17658822727785
max 12.607199668884277
min -2081.52001953125


In [76]:
"""1. Filter manually"""
# Filter z (dimension 1)
z_upper = -30
z_lower = -100 #removes ceiling points
filtered_points = point_cloud_np[(point_cloud_np[:, 1] < z_upper) & (point_cloud_np[:, 1] > z_lower)]

# Filter y (dimension 0)
y_upper = 80
y_lower = -780
filtered_points = filtered_points[(filtered_points[:, 0] < y_upper) & (filtered_points[:, 0] > y_lower)]

# Filter x (dimension 2)
x_upper = 1200
x_lower = 100
filtered_points = filtered_points[(filtered_points[:, 2] < x_upper) & (filtered_points[:, 2] > x_lower)]

In [77]:
"""Visualise"""
# transform to open3d pc again
filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)

o3d.visualization.draw_geometries([filtered_pcd]) 

In [104]:
"""2. Filter out isolated points / outliers with o3d function"""
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(filtered_points)

### statistical outlier filtering
pcd_clean, ind = pcd.remove_statistical_outlier(nb_neighbors=1200, std_ratio=2)
# nb_neighbors: Number of neighbors considered 
# Points further away than mean distance + std_ratio*std_dev are considered outliers

In [51]:
### radius filtering << didnt use
#pcd_clean, ind = pcd.remove_radius_outlier(nb_points=5, radius=1)
# radius: radius to consider for the neighbors
# min_points: minimum number of neighbors required within that radius

In [107]:
o3d.visualization.draw_geometries([pcd_clean])

In [106]:
"""Save filtered pointcloud"""
o3d.io.write_point_cloud("filtered_pointcloud_maze2.ply", pcd_clean)

True